In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import userdata
import os
os.environ["REPLICATE_API_TOKEN"]=userdata.get("REPLICATE_API_TOKEN")
!source ~/.bashrc

!pip install replicate
!pip install openai

import transformers
import torch
import pandas as pd
import replicate
from openai import OpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.7 MB/s eta 0:00:00


In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "meta-llama/Meta-Llama-3-70B-Instruct"

# model_id = "meta-llama/Llama-2-7b-chat-hf"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
def inference(prompt, cot=False):
    system_msg = "You answer questions by solely using logical reasoning. Only reply with 'TRUE', 'FALSE' or 'UNCERTAIN'. Do not write anything else."
    if cot:
        system_msg = "You answer questions by solely using logical reasoning. Explain how you arrive at your final answer using the argument forms provided."

    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": prompt},
    ]

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        messages,
        max_new_tokens=4000,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][-1]['content']

def inference_api(prompt, cot=False):
    system_msg = "You answer questions by solely using logical reasoning. Only reply with 'TRUE', 'FALSE' or 'UNCERTAIN'. Do not write anything else."
    if cot:
        system_msg = "You answer questions by solely using logical reasoning. Explain how you arrive at your final answer using the argument forms provided."

    completion = replicate.run(
        "meta/meta-llama-3-8b-instruct",
        input={
            "top_k": 0,
            "top_p": 0.9,
            "prompt": prompt,
            "max_tokens": 4000,
            "min_tokens": 0,
            "temperature": 0.6,
            "system_prompt": system_msg,
            "length_penalty": 1,
            "stop_sequences": "<|end_of_text|>,<|eot_id|>",
            "presence_penalty": 1.15,
            "log_performance_metrics": False
        }
    )
    return ''.join(completion)

def inference_openai(full_prompt, api_key, cot=False, stop=None, max_tokens=None, save_to_prolog_file=True):
    system_msg = "You answer questions by solely using logical reasoning. Only reply with 'TRUE', 'FALSE' or 'UNCERTAIN'. Do not write anything else."
    if cot:
        system_msg = "You answer questions by solely using logical reasoning. Explain how you arrive at your final answer using the argument forms provided."

    client = OpenAI(api_key=api_key)
    completion = client.chat.completions.create(
      model='gpt-4',
      messages=[
        {
            "role": "user",
            "content": full_prompt
        }
      ],
      stop=stop,
      max_tokens=max_tokens
    )
    result = completion.choices[0].message.content

    return completion, result

# inference('Can an elephant hide under a dining table?')

In [ ]:
def get_dataset(filename):
    df = pd.read_csv(filename)
    dataset = df.to_dict('records')
    return dataset

def get_prompt(filename):
    with open(filename, 'r', encoding='utf-8') as fr:
        prompt = fr.read()
    return prompt

def get_existing_results(filename):
    try:
        df = pd.read_csv(filename)
        dataset = df.to_dict('records')
    except:
        print('no existing results found.')
        dataset = []
    return dataset


In [ ]:
def main():
    results_file = '/content/drive/MyDrive/JustLogic/3_shot_cot_w_depth_gpt4_results.csv'
    dataset = get_dataset('/content/drive/MyDrive/JustLogic/test_dataset.csv')
    prompt = get_prompt('/content/drive/MyDrive/JustLogic/prompt_3_shot_cot_w_depth.txt')
    results = get_existing_results(results_file)
    cot = True
    api = False
    openai_api = True
    openai_api_key = userdata.get('OPENAI_API_KEY')

    for i, instance in enumerate(dataset):
        if instance['id'] in [r['id'] for r in results]:
            continue

        full_prompt = prompt.format(
            PARAGRAPH = instance['paragraph'],
            STATEMENT = instance['question']
        )
        if api == True:
            completion = inference_api(full_prompt, cot=cot)
        elif openai_api == True:
            _, completion = inference_openai(full_prompt, api_key=openai_api_key, cot=cot)
        else:
            completion = inference(full_prompt, cot=cot)

        *front, last_word = completion.split()
        if 'TRUE' in last_word or 'True' in last_word:
            completion_ans = True
        elif 'FALSE' in last_word or 'False' in last_word:
            completion_ans = False
        else:
            completion_ans = 'Uncertain'
        result = instance
        result['predicted'] = completion_ans
        if cot:
            result['full_completion'] = completion
        results.append(result)

        if i % 10 == 0:
            results_df = pd.DataFrame.from_records(results)
            results_df.to_csv(results_file, index=False)

        print('{i}: {ans}'.format(i=i, ans=completion_ans))

    results_df = pd.DataFrame.from_records(results)
    results_df.to_csv(results_file, index=False)

    return results

results = main()